In [ ]:
import torch
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
from torchvision.models import ResNet50_Weights, EfficientNet_B0_Weights, VGG16_Weights, Inception_V3_Weights, ViT_B_16_Weights
from torchvision import transforms, utils
from skimage import io, transform
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
from time import sleep
import re
from sklearn.metrics import precision_score, recall_score, accuracy_score, auc
from get_uncertainties import get_uncertainties
import warnings
warnings.filterwarnings("ignore")

In [ ]:
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

In [ ]:
class Classifier_output(torch.nn.Module):
    def __init__(self, num_classes, input_size):
        super(Classifier_output, self).__init__()
        
        self.fc = torch.nn.Sequential(torch.nn.Linear(input_size, 512), 
                               torch.nn.ReLU(inplace=True), 
                               torch.nn.Dropout(0.2),
                               torch.nn.Linear(512, 128), 
                               torch.nn.ReLU(inplace=True), 
                               torch.nn.Dropout(0.2),
                               torch.nn.Linear(128, 32), 
                               torch.nn.ReLU(inplace=True), 
                               torch.nn.Dropout(0.2),
                               torch.nn.Linear(32, num_classes))
        

    def forward(self, x):
        x = self.fc(x)
        
        return x

In [ ]:
model_name = 'ResNet50'
if model_name == 'ResNet50':
    model = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
    model.fc = Classifier_output(num_classes=30, input_size=2048)
    
    for param in model.parameters():
        param.requires_grad = False

    for param in model.fc.fc.parameters():
        param.requires_grad = True
    
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.fc.fc.parameters(), lr=0.001)
    
elif model_name == 'VGG16':
    model = models.vgg16(weights=VGG16_Weights.IMAGENET1K_V1)
    model.classifier[6] = Classifier_output(num_classes=30, input_size=4096)
    print(model)
    
    for param in model.parameters():
        param.requires_grad = False

    for param in model.classifier[6].parameters():
        param.requires_grad = True
    
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.classifier[6].parameters(), lr=0.001)
    
elif model_name == 'vit_b_16':
    model = models.vit_b_16(weights=ViT_B_16_Weights.IMAGENET1K_V1)
    model.heads = Classifier_output(num_classes=30, input_size=768)
    
    for param in model.parameters():
        param.requires_grad = False

    for param in model.heads.fc.parameters():
        param.requires_grad = True
    
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.heads.fc.parameters(), lr=0.001)
    
model.to(device)

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
class FacePix_Dataset(Dataset):

    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(os.listdir(self.root_dir))

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        img = os.listdir(self.root_dir)[idx]
        img_name = os.path.join(self.root_dir,
                                os.listdir(self.root_dir)[idx])
            
        image = cv2.imread(img_name)
        match = re.search(r'\((-?\d+)\)', img)
        if match:
            angle = int(match.group(1))
            if angle <= 5 and angle >= -5:
                angle = 0
            elif angle <= 20 and angle >= 10:
                angle = 1
            elif angle <= 35 and angle >= 25:
                angle = 2
            elif angle <= 50 and angle >= 40:
                angle = 3
            elif angle <= 65 and angle >= 55:
                angle = 4
            elif angle <= 80 and angle >= 70:
                angle = 5
        
        match = re.search(r'^(\d+)', img)
        if match:
            label = int(match.group(1)) - 1
        
        if self.transform:
            image = self.transform(image)

        return image, label, angle

In [ ]:
angle_0_dataset = FacePix_Dataset("D:\\jupyter_notebooks\\Experiments\\FacePix\dataset\\default_angle", transform=transform)

generator = torch.Generator().manual_seed(42)
angle_0_train_dataset, angle_0_valid_dataset = torch.utils.data.random_split(angle_0_dataset, [int(len(angle_0_dataset)*0.8), int(len(angle_0_dataset)*0.2)],generator=generator)
train_loader = torch.utils.data.DataLoader(angle_0_train_dataset, batch_size=16, shuffle=True)

In [ ]:
# Train the model
model.train()
num_epochs = 1
for epoch in range(num_epochs):
    loop = tqdm(train_loader)
    loss_epoch=0
    i=1
    for images, labels, angles in loop:
        loop.set_description(f"Epoch {epoch}")
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)
        angles = angles.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        loss_epoch += loss.item()
        loop.set_postfix(loss=loss.item())
        i +=1
      
    loss_epoch /= i
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss_epoch:.4f}')

In [ ]:
model_path = os.path.join("D:\\jupyter_notebooks\\Experiments\\FacePix\\models", f"facepix_model_{model_name}"+".pth")
#torch.save(model.state_dict(), model_path)
model.load_state_dict(torch.load(model_path))

In [ ]:
num_classes = 30
#evaluate on valid_dataset
with torch.no_grad():
    true_class = 0
    false_positives = np.zeros(num_classes)
    false_negatives = np.zeros(num_classes)
    true_positives = np.zeros(num_classes)
    model.eval()
    for i in range(len(angle_0_valid_dataset)):
        if int(i%100)==0:
            print(i)
        valid_img = torch.unsqueeze(angle_0_valid_dataset[i][0],0)
        valid_img = valid_img.to(device)
        pred = model(valid_img)
        pred_class = pred.argmax().item()
        
        # Update true positives, false positives, and false negatives for each class
        true_label = angle_0_valid_dataset[i][1]
        if pred_class == true_label:
            true_positives[true_label] += 1
        else:
            false_negatives[true_label] += 1
            false_positives[pred_class] += 1
            
    # Calculate precision and recall for each class
    precision = np.zeros(num_classes)
    recall = np.zeros(num_classes)
    for c in range(num_classes):
        precision[c] = true_positives[c] / (true_positives[c] + false_positives[c]) if (true_positives[c] + false_positives[c]) > 0 else 0
        recall[c] = true_positives[c] / (true_positives[c] + false_negatives[c]) if (true_positives[c] + false_negatives[c]) > 0 else 0

    # Calculate macro precision, recall, and accuracy
    macro_precision_default = np.mean(precision)
    macro_recall_default = np.mean(recall)
    macro_f1_score_default = 2*macro_precision_default*macro_recall_default/(macro_precision_default+macro_recall_default)
    macro_accuracy_default = np.sum(true_positives) / len(angle_0_valid_dataset)

    print("Accuracy: ", macro_accuracy_default)
    print("Precision: ", macro_precision_default)
    print("Recall: ", macro_recall_default)
    print("F1 Score: ", macro_f1_score_default)
    default_scores = [macro_accuracy_default,macro_precision_default,macro_recall_default,macro_f1_score_default]
    default_scores = np.reshape(np.array(default_scores),(1,4))

save_dir = 'default_scores'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
scores = ['Accuracy','Precision','Recall','F1 Score']
default_scores_df = pd.DataFrame(data=default_scores,columns=scores)
default_scores_df.to_csv(f'{save_dir}\\{model_name}.csv')

In [ ]:
for param in model.parameters():
    param.requires_grad = True

In [ ]:
#TTA_Dataset

In [ ]:
class TTA_Dataset(Dataset):

    def __init__(self, dataset, target_network, path):
        self.dataset = dataset
        self.target_network = target_network
        self.path = path

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        self.target_network.eval()

        image, label, angle = self.dataset[idx]
        image = torch.unsqueeze(image,0)
        image = image.to(device)
        
        TTA_lst = []
        TTA_lst_numpy = []
        for img2 in os.listdir(self.path):
            match = re.search(r'^(\d+)', img2)
            if match:
                img2_label = int(match.group(1)) - 1
                
            match_2 = re.search(r'\((-?\d+)\)', img2)
            if match_2:
                img2_angle = int(match_2.group(1))
            
            if label == img2_label and int((angle-img2_angle)%15)==0:
                tta_img = cv2.imread(os.path.join(self.path,img2))
                tta_img = transform(tta_img)
                tta_img = torch.unsqueeze(tta_img,0)
                TTA_lst_numpy.append(tta_img.numpy())
                tta_img = tta_img.to(device)
                TTA_lst.append(tta_img)
        
        tta_pred_lst = []
        uncertainty_lst = []
        loss_lst = []
        label = torch.ones(1,dtype=int)*label
        label = label.to(device)
        for tta_img in TTA_lst:
            tta_pred = self.target_network(tta_img)
            tta_pred_lst.append(torch.squeeze(tta_pred,0).cpu().detach().numpy())
            prediction, confidence, entropy, margin, NLL, Brier, ODIN_temp, MCD_temp, purview_initial_temp, purview_final_temp, grad_norm, grad_trust = get_uncertainties(tta_img,self.target_network)
            
            #uncertainties
            inv_confidence = 1/confidence[0]
            entropy = -entropy[0]
            inv_margin = 1/margin[0]
            NLL = NLL
            Brier = Brier
            ODIN_temp = ODIN_temp
            MCD_temp = -MCD_temp[0]
            purview_initial_temp = -purview_initial_temp
            purview_final_temp = -purview_final_temp
            grad_norm = -grad_norm
            grad_trust = grad_trust
            
            uncertainty = [inv_confidence,entropy,inv_margin,NLL,Brier,ODIN_temp,MCD_temp,purview_initial_temp,purview_final_temp,grad_norm,grad_trust]
            uncertainty_lst.append(uncertainty)
        tta_uncertainty = np.array(uncertainty_lst)
        
        #original
        pred = self.target_network(image)
        orig_prediction, orig_confidence, orig_entropy, orig_margin, orig_NLL, orig_Brier, orig_ODIN_temp, orig_MCD_temp, orig_purview_initial_temp, orig_purview_final_temp, orig_grad_norm, orig_grad_trust = get_uncertainties(image,self.target_network)
        
        #uncertainties
        orig_inv_confidence = 1/orig_confidence[0]
        orig_entropy = -orig_entropy[0]
        orig_inv_margin = 1/orig_margin[0]
        orig_NLL = orig_NLL
        orig_Brier = orig_Brier
        orig_ODIN_temp = orig_ODIN_temp
        orig_MCD_temp = -orig_MCD_temp[0]
        orig_purview_initial_temp = -orig_purview_initial_temp
        orig_purview_final_temp = -orig_purview_final_temp
        orig_grad_norm = -orig_grad_norm
        orig_grad_trust = orig_grad_trust
        
        orig_uncertainty = [orig_inv_confidence,orig_entropy,orig_inv_margin,orig_NLL,orig_Brier,orig_ODIN_temp,orig_MCD_temp,orig_purview_initial_temp,orig_purview_final_temp,orig_grad_norm,orig_grad_trust]
        orig_uncertainty = np.array(orig_uncertainty)
        
        image = torch.squeeze(image,0)
        image = image.cpu().detach().numpy()
        label = label.cpu().detach().numpy()[0]
        pred = pred.cpu().detach().numpy()[0]
        tta_pred_numpy = np.array(tta_pred_lst)
        
        return image, tta_uncertainty, label, TTA_lst_numpy, angle, orig_uncertainty, tta_pred_numpy, pred

In [ ]:
tta_dataset = TTA_Dataset(angle_0_train_dataset, model, "D:\\jupyter_notebooks\\Experiments\\FacePix\\dataset\\other_angles")
loss_predicter_loader = torch.utils.data.DataLoader(tta_dataset, batch_size=1, shuffle=True)

In [ ]:
#Find the best angle for every class-angle combination

In [ ]:
class_angle_arr = np.zeros((11,30,5))
for image, tta_uncertainty, label, TTA_lst_numpy, angle, orig_uncertainty, tta_pred_numpy, pred in tqdm(tta_dataset,ncols=100):
    best_angle = np.argmin(tta_uncertainty,axis=0)
    for i in range(class_angle_arr.shape[0]):
        class_angle_arr[i,label,best_angle[i]] += 1

np.save(f"class_angle\\{model_name}\\class_angle_facepix.npy",class_angle_arr)
class_angle_arr = np.load(f"class_angle\\{model_name}\\class_angle_facepix.npy")

In [ ]:
class_angle_arr = np.load(f"class_angle\\{model_name}\\class_angle_facepix.npy")
class_angle_best_lst = []
for i in range(class_angle_arr.shape[0]):
    class_angle = class_angle_arr[i,:,:]
    class_angle_best = np.argsort(class_angle, axis=1)[:,-1]
    class_angle_second_best = np.argsort(class_angle, axis=1)[:,-2]
    class_angle_best_second_best = np.array([class_angle_best,class_angle_second_best])
    class_angle_best_lst.append(class_angle_best_second_best)
class_angle_best_arr = np.array(class_angle_best_lst)

In [ ]:
#saving arrays

In [ ]:
tta_dataset_valid = TTA_Dataset(angle_0_valid_dataset, model, "D:\\jupyter_notebooks\\Experiments\\FacePix\\dataset\\other_angles")
pred_lst = []
tta_pred_lst = []
orig_uncertainty_lst = []
tta_uncertainty_lst = []
for image, tta_uncertainty, label, TTA_lst_numpy, angle, orig_uncertainty, tta_pred_numpy, pred in tqdm(tta_dataset_valid,ncols=100):
    pred_lst.append(pred)
    tta_pred_lst.append(tta_pred_numpy)
    orig_uncertainty_lst.append(orig_uncertainty)
    tta_uncertainty_lst.append(tta_uncertainty)

np.save(f"numpy_arrays\\{model_name}\\pred.npy",np.array(pred_lst))
np.save(f"numpy_arrays\\{model_name}\\tta_pred.npy",np.array(tta_pred_lst))
np.save(f"numpy_arrays\\{model_name}\\orig_uncertainty.npy",np.array(orig_uncertainty_lst))
np.save(f"numpy_arrays\\{model_name}\\tta_uncertainty.npy",np.array(tta_uncertainty_lst))

In [ ]:
#accuracy, precision, recall, f1-score

In [ ]:
pred_array = np.load(f"numpy_arrays\\{model_name}\\pred.npy")
tta_pred_array = np.load(f"numpy_arrays\\{model_name}\\tta_pred.npy")
orig_uncertainty_array = np.load(f"numpy_arrays\\{model_name}\\orig_uncertainty.npy")
tta_uncertainty_array = np.load(f"numpy_arrays\\{model_name}\\tta_uncertainty.npy")
max_uncertainty = np.nanmax(orig_uncertainty_array,axis=0)
min_uncertainty = np.nanmin(orig_uncertainty_array,axis=0)
print(pred_array.shape)
print(tta_pred_array.shape)
print(orig_uncertainty_array.shape)
print(tta_uncertainty_array.shape)

In [ ]:
orig_uncertainty_mean = np.nanmean(orig_uncertainty_array,axis=0)
tta_uncertainty_mean = np.nanmean(tta_uncertainty_array,axis=0)
uncertainty_mean = np.concatenate((np.reshape(orig_uncertainty_mean,(1,11)),tta_uncertainty_mean),axis=0)
uncertainties = ['inv_confidence','entropy','inv_margin','NLL','Brier','ODIN_temp','MCD_temp','purview_initial_temp','purview_final_temp','grad_norm','grad_trust']
angle_names = ['Default Angle', 'Augmentation Angle 1', 'Augmentation Angle 2', 'Augmentation Angle 3', 'Augmentation Angle 4', 'Augmentation Angle 5']
pd.options.display.float_format = '{:,.4f}'.format
uncertainty_df = pd.DataFrame(data=uncertainty_mean,columns=uncertainties,index=angle_names)
uncertainty_df.to_csv(f'numpy_arrays\\{model_name}\\uncertainty_mean_table.csv')
uncertainty_df

In [ ]:
#Parameter sweep

In [ ]:
save_dir = f'parameter_sweep_results\\{model_name}\\parameter_sweep_results_method_2'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
    
uncertainties = ['inv_confidence','entropy','inv_margin','NLL','Brier','ODIN_temp','MCD_temp','purview_initial_temp','purview_final_temp','grad_norm','grad_trust']
tta_algorithms = ['Random Angle', 'Best Angle', 'Default + Best Angle', 'Default + 2 Best Angles']
for u,uncertainty in enumerate(uncertainties):
    for alg,tta_algorithm in enumerate(tta_algorithms):
        algorithm = tta_algorithm + ' ' + uncertainty
        threshold_bins = np.linspace(min_uncertainty[u], max_uncertainty[u], 11)
        class_accuracy_lst = []
        class_precision_lst = []
        class_recall_lst = []
        class_f1_score_lst = []

        num_classes = pred_array.shape[1]
        with torch.no_grad():
            model.eval()
            for j, uncertainty_threshold in enumerate(threshold_bins):
                if j==10:
                    print(j)
                true_class = 0
                false_positives = np.zeros(num_classes)
                false_negatives = np.zeros(num_classes)
                true_positives = np.zeros(num_classes)

                for i in range(len(angle_0_valid_dataset)):
                    orig_uncertainty = orig_uncertainty_array[i,u] 
                    first_pred = pred_array[i,:]
                    first_pred_class = first_pred.argmax().item()

                    if orig_uncertainty < uncertainty_threshold:
                        pred_class = first_pred_class
                    else:
                        if alg==0:
                            best_idx = np.random.randint(0,5)
                            pred = tta_pred_array[i,best_idx,:]
                        elif alg==1:
                            best_idx = class_angle_best_arr[u,0,first_pred_class]
                            pred = tta_pred_array[i,best_idx,:]
                        elif alg==2:
                            best_idx = class_angle_best_arr[u,0,first_pred_class]
                            pred_aug = tta_pred_array[i,best_idx,:]
                            pred = (first_pred + pred_aug)/2
                        elif alg==3:
                            best_idx = class_angle_best_arr[u,0,first_pred_class]
                            second_best_idx = class_angle_best_arr[u,1,first_pred_class]
                            pred_aug = tta_pred_array[i,best_idx,:]
                            second_pred_aug = tta_pred_array[i,second_best_idx,:]
                            pred = (first_pred + pred_aug + second_pred_aug)/3
      
                        pred_class = pred.argmax().item()

                    # Update true positives, false positives, and false negatives for each class
                    true_label = angle_0_valid_dataset[i][1]
                    if pred_class == true_label:
                        true_positives[true_label] += 1
                    else:
                        false_negatives[true_label] += 1
                        false_positives[pred_class] += 1

                # Calculate precision and recall for each class
                precision = np.zeros(num_classes)
                recall = np.zeros(num_classes)
                for c in range(num_classes):
                    precision[c] = true_positives[c] / (true_positives[c] + false_positives[c]) if (true_positives[c] + false_positives[c]) > 0 else 0
                    recall[c] = true_positives[c] / (true_positives[c] + false_negatives[c]) if (true_positives[c] + false_negatives[c]) > 0 else 0

                # Calculate macro precision, recall, and accuracy
                macro_precision = np.mean(precision)
                macro_recall = np.mean(recall)
                macro_f1_score = 2*macro_precision*macro_recall/(macro_precision+macro_recall)
                macro_accuracy = np.sum(true_positives) / len(angle_0_valid_dataset)

                class_accuracy_lst.append(macro_accuracy)
                class_precision_lst.append(macro_precision)
                class_recall_lst.append(macro_recall)
                class_f1_score_lst.append(macro_f1_score)

        # Save or plot your results as needed
        np.save(f"{save_dir}\\{algorithm}_accuracies.npy", np.array(class_accuracy_lst))
        np.save(f"{save_dir}\\{algorithm}_precisions.npy", np.array(class_precision_lst))
        np.save(f"{save_dir}\\{algorithm}_recalls.npy", np.array(class_recall_lst))
        np.save(f"{save_dir}\\{algorithm}_f1_score.npy", np.array(class_f1_score_lst))

#plot accuracy
for u,uncertainty in enumerate(uncertainties):
    plt.figure()
    for alg,tta_algorithm in enumerate(tta_algorithms):
        algorithm = tta_algorithm + ' ' + uncertainty
        accuracy_array = np.load(f"{save_dir}\\{algorithm}_accuracies.npy")
        plt.plot(np.linspace(1, 11, 11), accuracy_array, label=algorithm)
    plt.plot(np.linspace(1, 11, 11), np.repeat(macro_accuracy_default,11), linestyle='dashed', label='Test Accuracy')
    plt.xlabel('Uncertainty Threshold')
    plt.ylabel('Test Accuracy')
    plt.title(f'Test Accuracy vs Uncertainty Threshold ({uncertainty})')
    plt.legend()
    plt.savefig(f'{save_dir}\\Test Accuracy vs Uncertainty Threshold ({uncertainty}).jpg')

#plot precision
for u,uncertainty in enumerate(uncertainties):
    plt.figure()
    for alg,tta_algorithm in enumerate(tta_algorithms):
        algorithm = tta_algorithm + ' ' + uncertainty
        precision_array = np.load(f"{save_dir}\\{algorithm}_precisions.npy")
        plt.plot(np.linspace(1, 11, 11), precision_array, label=algorithm)
    plt.plot(np.linspace(1, 11, 11), np.repeat(macro_precision_default,11), linestyle='dashed', label='Test Precision')
    plt.xlabel('Uncertainty Threshold')
    plt.ylabel('Test Precision')
    plt.title(f'Test Precision vs Uncertainty Threshold ({uncertainty})')
    plt.legend()
    plt.savefig(f'{save_dir}\\Test Precision vs Uncertainty Threshold ({uncertainty}).jpg')

#plot recall
for u,uncertainty in enumerate(uncertainties):
    plt.figure()
    for alg,tta_algorithm in enumerate(tta_algorithms):
        algorithm = tta_algorithm + ' ' + uncertainty
        recall_array = np.load(f"{save_dir}\\{algorithm}_recalls.npy")
        plt.plot(np.linspace(1, 11, 11), recall_array, label=algorithm)
    plt.plot(np.linspace(1, 11, 11), np.repeat(macro_recall_default,11), linestyle='dashed', label='Test Recall')
    plt.xlabel('Uncertainty Threshold')
    plt.ylabel('Test Recall')
    plt.title(f'Test Recall vs Uncertainty Threshold ({uncertainty})')
    plt.legend()
    plt.savefig(f'{save_dir}\\Test Recall vs Uncertainty Threshold ({uncertainty}).jpg')

#plot f1-score
for u,uncertainty in enumerate(uncertainties):
    plt.figure()
    for alg,tta_algorithm in enumerate(tta_algorithms):
        algorithm = tta_algorithm + ' ' + uncertainty
        f1_score_array = np.load(f"{save_dir}\\{algorithm}_f1_score.npy")
        plt.plot(np.linspace(1, 11, 11), f1_score_array, label=algorithm)
    plt.plot(np.linspace(1, 11, 11), np.repeat(macro_f1_score_default,11), linestyle='dashed', label='Test F1-Score')
    plt.xlabel('Uncertainty Threshold')
    plt.ylabel('Test F1-Score')
    plt.title(f'Test F1-Score vs Uncertainty Threshold ({uncertainty})')
    plt.legend()
    plt.savefig(f'{save_dir}\\Test F1-Score vs Uncertainty Threshold ({uncertainty}).jpg')
    
#calculate normalized auc
with open(f'{save_dir}\\auc_results.txt', 'w') as file:
    for u, uncertainty in enumerate(uncertainties):
        best_auc = 0
        best_tta_algorithm = None
        file.write(uncertainty + '\n')
        for alg, tta_algorithm in enumerate(tta_algorithms):
            algorithm = tta_algorithm + ' ' + uncertainty
            accuracy_array = np.load(f"{save_dir}\\{algorithm}_accuracies.npy")
            auc_value = auc(np.linspace(min_uncertainty[u], max_uncertainty[u], 11), accuracy_array)
            normalized_auc = auc_value/(max_uncertainty[u]-min_uncertainty[u])
            if normalized_auc > best_auc:
                best_auc = normalized_auc
                best_tta_algorithm = tta_algorithm
            file.write(tta_algorithm + ': ' + str(normalized_auc) + '\n')
        file.write('Best Algorithm: '+ best_tta_algorithm + '\n')
        file.write('\n')

In [ ]:
#Creating auc table, finding max accuracy

In [ ]:
#calculate normalized auc
save_dir = f'parameter_sweep_results\\{model_name}\\parameter_sweep_results'
uncertainties = ['inv_confidence','entropy','inv_margin','NLL','Brier','ODIN_temp','MCD_temp','purview_initial_temp','purview_final_temp','grad_norm','grad_trust']
tta_algorithms = ['Random Angle', 'Best Angle', 'Default + Best Angle', 'Default + 2 Best Angles']
auc_df = pd.DataFrame(data=np.zeros((4,11)),columns=uncertainties,index=tta_algorithms)
max_acc_df = pd.DataFrame(data=np.zeros((4,11)),columns=uncertainties,index=tta_algorithms)
arg_max_acc_df = pd.DataFrame(data=np.zeros((4,11)),columns=uncertainties,index=tta_algorithms)
pd.options.display.float_format = '{:,.2f}'.format
with open(f'{save_dir}\\auc_results.txt', 'w') as file:
    for u, uncertainty in enumerate(uncertainties):
        best_auc = 0
        best_tta_algorithm = None
        file.write(uncertainty + '\n')
        for alg, tta_algorithm in enumerate(tta_algorithms):
            algorithm = tta_algorithm + ' ' + uncertainty
            accuracy_array = np.load(f"{save_dir}\\{algorithm}_accuracies.npy")
            max_acc_df.iloc[alg,u] = np.max(accuracy_array)
            arg_max_acc_df.iloc[alg,u] = np.argmax(accuracy_array)
            auc_value = auc(np.linspace(min_uncertainty[u], max_uncertainty[u], 11), accuracy_array)
            normalized_auc = auc_value/(max_uncertainty[u]-min_uncertainty[u])
            auc_df.iloc[alg,u] = normalized_auc
            if normalized_auc > best_auc:
                best_auc = normalized_auc
                best_tta_algorithm = tta_algorithm
            file.write(tta_algorithm + ': ' + str(normalized_auc) + '\n')
        file.write('Best Algorithm: '+ best_tta_algorithm + '\n')
        file.write('\n')
auc_df.to_csv(f'{save_dir}\\auc_table.csv')
max_acc_df.to_csv(f'{save_dir}\\max_acc_table.csv')
arg_max_acc_df.to_csv(f'{save_dir}\\arg_max_acc_table.csv')

highlight_threshold = macro_accuracy_default  # You can adjust this threshold as needed

# Function to make the element bold if it is higher than the threshold
def highlight_max_with_threshold(s, threshold):
    is_higher_than_threshold = s > threshold
    return ['font-weight: bold' if v else '' for v in is_higher_than_threshold]

# Apply the styling to the DataFrame with the specified threshold
bold_auc_df = auc_df.style.apply(highlight_max_with_threshold, threshold=highlight_threshold).format('{:.2f}')
bold_max_acc_df = max_acc_df.style.apply(highlight_max_with_threshold, threshold=highlight_threshold).format('{:.2f}')
bold_max_acc_df